# YT Comments analysis

In [1]:
from googleapiclient.discovery import build
import pandas as pd
from api import API_KEY

channel_id = "UCWeg2Pkate69NFdBeuRFTAw" #Squeezie channel

youtube = build('youtube', 'v3', developerKey=API_KEY)

exemple_video = "br-Ao33SfMw"

/usr/lib/python3/dist-packages/pyasn1/type/constraint.py:100: SyntaxWarning: invalid escape sequence '\*'
  """Create a SingleValueConstraint object.
/usr/lib/python3/dist-packages/pyasn1/type/constraint.py:141: SyntaxWarning: invalid escape sequence '\*'
  """Create a ContainedSubtypeConstraint object.
/usr/lib/python3/dist-packages/pyasn1/type/constraint.py:301: SyntaxWarning: invalid escape sequence '\*'
  """Create a PermittedAlphabetConstraint object.
/usr/lib/python3/dist-packages/pyasn1/type/constraint.py:457: SyntaxWarning: invalid escape sequence '\*'
  """Create a ConstraintsIntersection logic operator object.
/usr/lib/python3/dist-packages/pyasn1/type/constraint.py:501: SyntaxWarning: invalid escape sequence '\*'
  """Create a ConstraintsUnion logic operator object.
/usr/lib/python3/dist-packages/pyasn1/type/namedval.py:15: SyntaxWarning: invalid escape sequence '\*'
  """Create named values object.
/tmp/ipykernel_266752/3782857391.py:2: DeprecationWarning: 
Pyarrow will bec

In [2]:
def get_channel_stats(youtube, channel_id):
    request = youtube.channels().list(
        part="snippet,contentDetails,statistics",
        id=channel_id
    )
    response = request.execute()
    data = {
        "channel_name": response['items'][0]['snippet']['title'],
        "channel_id": response['items'][0]['id'],
        "stats": response['items'][0]['statistics']
    }
    return data

In [3]:
def get_video_comments(youtube,video_Id):
    request = youtube.commentThreads().list(
        part="snippet",
        videoId=video_Id,
        maxResults=1000,
    )
    response = request.execute()
    return response

In [4]:
def get_video_comments_texts(youtube, video_Id):
    request = youtube.commentThreads().list(
        part="snippet",
        videoId=video_Id,
        maxResults=1150,
    )
    response = request.execute()
    
    # Extract textOriginal from each item
    comments_texts = []
    for item in response.get("items", []):
        if "snippet" in item and "topLevelComment" in item["snippet"]:
            comment_snippet = item["snippet"]["topLevelComment"]["snippet"]
            if "textOriginal" in comment_snippet:
                comments_texts.append(comment_snippet["textOriginal"])
    
    return comments_texts

In [5]:
get_channel_stats(youtube, channel_id)
video = get_video_comments_texts(youtube, exemple_video)

In [6]:
pd.set_option('display.max_rows', False)
df = pd.DataFrame({"Com":video})

df

,Com
0,"I haven't been watching you for 4 years, damn ..."
1,Your son seems really intelligent. Im so happy...
2,W Dog
3,"i love pewds vlogs with his family, it's so he..."
4,very cuteeeeeee😢
5,"8:50\nScreanshotted it, 10 $ on feet finder"
6,Bjorn looks soooooo cute 🥺
7,who loves pew die pie?
8,🤍❤️‍🩹👑
...,...


In [7]:
from langdetect import detect, LangDetectException

def detect_language(text):
    try:
        return detect(text)
    except LangDetectException:
        return "unknown"

df["Language"] = df["Com"].apply(detect_language)
df = df.loc[df['Language'] == 'en']
df

,Com,Language
0,"I haven't been watching you for 4 years, damn ...",en
1,Your son seems really intelligent. Im so happy...,en
3,"i love pewds vlogs with his family, it's so he...",en
5,"8:50\nScreanshotted it, 10 $ on feet finder",en
7,who loves pew die pie?,en
9,Bjorn is such a blessing 🤍 such BEAUTIFUL baby...,en
10,cant believe we are uncle's now,en
11,I haven’t seen a pewdiepie video in five years...,en
12,Thank you for enjoying Kamakura! Kamakura is a...,en
...,...,...


In [8]:
import nltk

from nltk.sentiment.vader import SentimentIntensityAnalyzer

from nltk.corpus import stopwords

from nltk.tokenize import word_tokenize

from nltk.stem import WordNetLemmatizer

import re

In [9]:
### A faire la premiere fois ###
#nltk.download('all')

In [13]:
def remove_punctuation(text):
    # Utilisation d'une expression régulière pour supprimer les caractères de ponctuation
    return re.sub(r'[^\w\s]', '', text)
df['No_ponctuation'] = df['Com'].apply(remove_punctuation)
df

/tmp/ipykernel_266752/2591894868.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['No_ponctuation'] = df['Com'].apply(remove_punctuation)


,Com,Language,No_ponctuation,Preprocessing
0,"I haven't been watching you for 4 years, damn ...",en,I havent been watching you for 4 years damn wh...,havent watching 4 year damn missed ! ! look .....
1,Your son seems really intelligent. Im so happy...,en,Your son seems really intelligent Im so happy ...,son seems really intelligent . im happy guy . ...
3,"i love pewds vlogs with his family, it's so he...",en,i love pewds vlogs with his family its so hear...,love pewds vlogs family heartwarming🥺♥️♥️♥️♥️
5,"8:50\nScreanshotted it, 10 $ on feet finder",en,850\nScreanshotted it 10 on feet finder,8:50 screanshotted 10 $ foot finder
7,who loves pew die pie?,en,who loves pew die pie,love pew die pie ?
9,Bjorn is such a blessing 🤍 such BEAUTIFUL baby...,en,Bjorn is such a blessing such BEAUTIFUL babyy...,bjorn blessing 🤍 beautiful babyyy ✨✨✨ always s...
10,cant believe we are uncle's now,en,cant believe we are uncles now,cant believe uncle
11,I haven’t seen a pewdiepie video in five years...,en,I havent seen a pewdiepie video in five years ...,’ seen pewdiepie video five year 🥲 kinda going...
12,Thank you for enjoying Kamakura! Kamakura is a...,en,Thank you for enjoying Kamakura Kamakura is an...,thank enjoying kamakura ! kamakura ancient cit...
...,...,...,...,...


In [14]:
def preprocess(text):
    emoji_pattern = re.compile("["
        #u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags=re.UNICODE)
    text_no_emojis = emoji_pattern.sub(r'', text)
    tokens = word_tokenize(text_no_emojis.lower())
    filtered_tokens = [token for token in tokens if token not in stopwords.words('english')]
    lemmatizer = WordNetLemmatizer()
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in filtered_tokens]
    processed_text = ' '.join(lemmatized_tokens)
    return processed_text

df["Preprocessing"] = df["No_ponctuation"].apply(preprocess)
df

/tmp/ipykernel_266752/2602301882.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Preprocessing"] = df["No_ponctuation"].apply(preprocess)


,Com,Language,No_ponctuation,Preprocessing
0,"I haven't been watching you for 4 years, damn ...",en,I havent been watching you for 4 years damn wh...,havent watching 4 year damn missed lookelder s...
1,Your son seems really intelligent. Im so happy...,en,Your son seems really intelligent Im so happy ...,son seems really intelligent im happy guy good...
3,"i love pewds vlogs with his family, it's so he...",en,i love pewds vlogs with his family its so hear...,love pewds vlogs family heartwarming
5,"8:50\nScreanshotted it, 10 $ on feet finder",en,850\nScreanshotted it 10 on feet finder,850 screanshotted 10 foot finder
7,who loves pew die pie?,en,who loves pew die pie,love pew die pie
9,Bjorn is such a blessing 🤍 such BEAUTIFUL baby...,en,Bjorn is such a blessing such BEAUTIFUL babyy...,bjorn blessing beautiful babyyy always smiling...
10,cant believe we are uncle's now,en,cant believe we are uncles now,cant believe uncle
11,I haven’t seen a pewdiepie video in five years...,en,I havent seen a pewdiepie video in five years ...,havent seen pewdiepie video five year kinda go...
12,Thank you for enjoying Kamakura! Kamakura is a...,en,Thank you for enjoying Kamakura Kamakura is an...,thank enjoying kamakura kamakura ancient city ...
...,...,...,...,...
